In [1]:
!pip install h2o

     |████████████████████████████████| 177.0 MB 17 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=778dba998df4a8a07d8b58a708d1148aaf938a654f07ac01d575f02ac99fef9d
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [2]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpcvi8s_c9
  JVM stdout: /tmp/tmpcvi8s_c9/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpcvi8s_c9/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_unknownUser_rfbjwk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [5]:
test, train = dataSetup(225)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
train.shape

(6144, 9)

In [7]:
x = train.columns
y = "class"
x.remove(y)

In [8]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [9]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |██
05:56:50.583: XGBoost_1_AutoML_1_20220514_55638 [XGBoost def_2] failed: java.lang.RuntimeException: Error while training XGBoost model

████████
05:58:06.756: GLM_1_AutoML_1_20220514_55638 [GLM def_1] failed: java.lang.UnsupportedOperationException: error cannot be computed: too many classes

█
05:58:20.828: GBM_1_AutoML_1_20220514_55638 [GBM def_5] failed: java.lang.UnsupportedOperationException: error cannot be computed: too many classes

█
05:58:27.974: XGBoost_2_AutoML_1_20220514_55638 [XGBoost def_1] failed: java.lang.RuntimeException: Error while training XGBoost model


05:58:32.548: DRF_1_AutoML_1_20220514_55638 [DRF def_1] failed: java.lang.UnsupportedOperationException: error cannot be computed: too many classes

██
05:58:44.895: GBM_2_AutoML_1_20220514_55638 [GBM def_2] failed: java.lang.UnsupportedOperationException: error cannot be computed: too many classes

█
05:58:57.643: GBM_3_AutoML_1_20220514_55638 [GBM def_3] failed: java.lang.UnsupportedOperati

OSError: ignored

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)